In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd

In [31]:
# global variables

l1, l2, l3, l4, l5, l6 = [],[],[],[],[],[]
links=[]
UniLinks=[]
op = Options()
op.add_argument('--headless')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=op)

In [32]:
def get_soup(flink, showAll = False, d = False):
    driver.get(flink)
    if showAll:
        try:
            driver.find_element(By.CLASS_NAME,"showAlllink").click()
        except:
            pass
    driver.implicitly_wait(10)
    soupfile = BeautifulSoup(driver.page_source, 'html.parser')
    if d:
        driver.close()
    print("Soup Recieved")
    
    return soupfile

In [33]:
print("Extracting Employability Rate")

EmpRateLink = "https://www.gooduniversitiesguide.com.au/university-ratings-rankings/2023/undergraduate/full-time-employment"
soupy = get_soup(EmpRateLink)

AllEmpRate = soupy.find_all('div', attrs={'class' : 'row mt-2 mb-3 pt-3 pb-3 bg-white border-bottom'})       

uni_abv = {'Macquarie University':'mq','Western Sydney University':'ws','University of Sydney':'usyd','University of Technology Sydney (UTS)':'uts','UNSW Sydney':'unsw'}

emp_rate = {"mq":None,"ws":None,"usyd":None, "uts":None, "unsw":None}

for x in AllEmpRate:
    for uni in ['Macquarie University','Western Sydney University','University of Sydney','University of Technology Sydney (UTS)','UNSW Sydney']:
        if x.find('h5', attrs = {'class': 'mb-0 font-weight-bold'}).text.strip() == uni:
            emp_rate[uni_abv[uni]] = x.find('span', attrs={'class' : 'mt-3 h5 theme-color-2'}).text

print(emp_rate)

Extracting Employability Rate
Soup Recieved
{'mq': ' 68.8% ', 'ws': ' 63.2% ', 'usyd': ' 75.5% ', 'uts': ' 72.9% ', 'unsw': ' 77% '}


In [34]:
search = input("Enter the course name you want:").rstrip().replace(" ","+")
UAC_link= f'https://www.uac.edu.au/course-search/undergraduate/find-a-course.html?search={search}'
soup =get_soup(UAC_link,showAll = True)
course_cards= soup.find_all('div', attrs={'class' : 'col course-left-col'})

Soup Recieved


In [35]:
for a in course_cards:
    for h in a.find_all('a', href=True):
        UniLinks.append(h['href'])
        break

for uni in UniLinks:
    for unifil in emp_rate.keys():
        if unifil in uni:
                links.append(uni)
                l6.append(emp_rate[unifil])

In [36]:
for lnk in links:
    soup=get_soup(lnk)
    l1.append(soup.find('h1', attrs={'class' : 'course-title'}).text)
    l2.append(soup.find('p', attrs={'class' : 'institution-name'}).text)
    l3.append(soup.find('span', attrs={'class' : 'course-location-campus mt-0 mob-table-cell ng-binding'}).text)
    l4.append(soup.find('div', attrs={'id' : 'admission'}).text.split("\n")[3].split(":")[1].strip())
    l5.append(pd.read_html(str(soup.find('table', {'id': 'atarDataTable'})))[0]['Lowest ATAR'][0])

driver.close()
dict={"Course Name":l1, "University Name": l2,"Course Location": l3,"Admission Criteria":l4,"ATAR":l5, "Employability Rate":l6 }
df=pd.DataFrame(dict)
df.to_csv("project.csv")
df

Soup Recieved
Soup Recieved
Soup Recieved


,Course Name,University Name,Course Location,Admission Criteria,ATAR,Employability Rate
0,Bachelor of Social Work,University of Sydney,Camperdown/Darlington,Depends on first-year subjects chosen.,70.15,75.5%
1,Bachelor of Arts/Bachelor of Social Work,University of Sydney,Camperdown/Darlington,Depends on first-year subjects chosen.,73.75,75.5%
2,International Studies Combined Courses,University of Technology Sydney,City,No prior language knowledge is required; refer...,76.75,72.9%
